Sandy Tsan, 861299012

Douglas Tran, 861208900

# Riverside Crime Reports Cleaning
### Part 1: Dataset info 

In [7]:
import pandas as pd
import numpy as np
crimes_df = pd.read_csv("Crime_Reports.csv")
crimes_df = crimes_df[["offenseDate","crimeType","premise","blockAddress"]].copy()
crimes_df['offenseDate'].replace('', np.nan, inplace=True)
crimes_df = crimes_df.dropna(subset=['offenseDate'])

restaurants_df = pd.read_csv('Yelp/restaurants.csv')
restaurants_df = restaurants_df.replace(np.nan, 0.0, regex=True)
restaurants_df.head()

,name,price,rating,review_count,address,city,state,zip_code
0,Diablos Micheladas Mexican Grill,$$,4.0,19,8304 Limonite Ave,Riverside,CA,92509
1,Sushi R91,$$,4.0,417,1630 Spruce St,Riverside,CA,92507
2,Placita,$$,4.0,182,1805 University Ave,Riverside,CA,92507
3,Rockstar Shrimp,$$,4.0,434,767 W Blaine St,Riverside,CA,92507
4,Yard House,$$,3.5,876,3775 Tyler St,Riverside,CA,92503


In [8]:
crimes_df.head()

,offenseDate,crimeType,premise,blockAddress
0,12/3/2018,MOTOR VEH. THEFT,RESIDENTIAL,4700 BLOCK BEVERLY CT
1,12/4/2018,ROBBERY,RESIDENTIAL,9300 BLOCK INDIANA AVE
2,12/4/2018,VANDALISM,MISCELLANEOUS,3600 BLOCK NYE AVE
3,12/4/2018,OTHER,HIGHWAYS,GOLDEN AVE/ TURNINGBEN WAY
4,12/3/2018,ASSAULT,RESIDENTIAL,2500 BLOCK 14TH ST


### Part 2: Building a model
This can include building a model to perform prediction (like applying linear regression or kNN) or clustering. 

You can also use the models for data analysis, not just ‘predictions’. For example, in linear regression we saw that the
resulting coefficients tell us how the features are correlated to the target variable. 

So, this analysis might help you identify features of importance with respect to a target feature in the dataset.



### Part 3: Data analysis
For analysis, identify two hypotheses (or questions) you are curious about regarding your data
and then proceed to use ML techniques to try to answer these questions.

If you decide to build
models for predictions or classification, be sure to setup your problem so that you have a training,
validation and testing dataset and you evaluate at least two ML / Data Mining techniques and
compare their results.